# RUN: SARS-CoV-2 Zoonotic Reservoir V Clean-up

```
Lead     : ababaian
Issue    : #55
Version  : 
start    : 2020 05 13
complete : YYYY MM DD
files    : ~/serratus/notebook/200505_ab/
s3_files : s3://serratus-public/notebook/200505_ab/
output   : s3://serratus-public/out/200505_zoonotic/
```

Continuation from `200505_Run_Zoonotic_Reservoir.ipynb`

UPDATE TO `cov2m` genome.

In [1]:
date

Wed May 13 11:05:18 PDT 2020


### Initialize local workspace

In [2]:
# Serratus commit version
SERRATUS="/home/artem/serratus"
cd $SERRATUS

## Serratus was updated, genome remains the same
git rev-parse HEAD # commit version

# Create local run directory
WORK="$SERRATUS/notebook/200505_ab"
mkdir -p $WORK; cd $WORK

# SRA RunInfo Table for run -- use first 500 from Zoonotic pilot
RUNINFO="$SERRATUS/notebook/200505_ab/zoonotic_SraRunInfo.csv"

913c45f0a5e0052d4a16fae5680cc7f3d7742d30


In [4]:
# Create a list of all completed runs to date
cd $WORK
aws s3 ls s3://serratus-public/out/200505_zoonotic/summary/ > batchC.complete
cat batchC.complete | sed 's/^...............................//g' - | cut -f1 -d'.' - > batchC.sra.complete

wc -l zoonotic_SraRunInfo.csv
wc -l batchC.sra.complete

sed -i 's/$/,/g' batchC.sra.complete 
head -n2 batchC.sra.complete

# Generate an updated RunInfo File with all matches NOT completed
# This is dropping ~50 entries I'll need to figure out later
grep -vif batchC.sra.complete zoonotic_SraRunInfo.csv > zoonotic_SraRunInfo_Batch14.csv

wc -l zoonotic_SraRunInfo_Batch14.csv

70966 zoonotic_SraRunInfo.csv
65202 batchC.sra.complete
DRR003377,
DRR003378,
5610 zoonotic_SraRunInfo_Batch14.csv


In [5]:
# Create a list of all completed runs to date
CURRENT_BATCH="zoonotic_SraRunInfo_Batch14.csv"

head -n 5 $CURRENT_BATCH

Run,ReleaseDate,LoadDate,spots,bases,spots_with_mates,avgLength,size_MB,AssemblyName,download_path,Experiment,LibraryName,LibraryStrategy,LibrarySelection,LibrarySource,LibraryLayout,InsertSize,InsertDev,Platform,Model,SRAStudy,BioProject,Study_Pubmed_id,ProjectID,Sample,BioSample,SampleType,TaxID,ScientificName,SampleName,g1k_pop_code,source,g1k_analysis_group,Subject_ID,Sex,Disease,Tumor,Affection_Status,Analyte_Type,Histological_Type,Body_Site,CenterName,Submission,dbgap_study_accession,Consent,RunHash,ReadHash
ERR3890427,2020-04-01 23:45:50,,0,0,0,0,0,,,ERX3900663,unspecified,RNA-Seq,Oligo-dT,TRANSCRIPTOMIC,PAIRED,480,0,ILLUMINA,Illumina NovaSeq 6000,ERP119658,PRJEB36462,,616001,ERS4294235,SAMEA6529835,simple,9796,Equus caballus,H35_C_Ep,,,,,,,no,,,,,UNIVERSITY OF BERN,ERA2378032,,public,,
ERR3890429,2020-04-01 23:45:50,,0,0,0,0,0,,,ERX3900665,unspecified,RNA-Seq,Oligo-dT,TRANSCRIPTOMIC,PAIRED,482,0,ILLUMINA,Illumina NovaSeq 6000,ERP119658,PRJEB36462,,616001,ERS4294237,SAMEA6529837

### Terraform Initialization



In [6]:
# Terraform customization
# Make scheduler/monitor beefier for more nodes
git diff $SERRATUS/terraform/main/main.tf

diff --git a/terraform/main/main.tf b/terraform/main/main.tf
index 606aa6c..80b3c5f 100644
--- a/terraform/main/main.tf
+++ b/terraform/main/main.tf
@@ -89,7 +89,7 @@ module "scheduler" {
   
   security_group_ids = [aws_security_group.internal.id]
   key_name           = var.key_name
-  instance_type      = "t3.small"
+  instance_type      = "c5.large"
   dockerhub_account  = var.dockerhub_account
   scheduler_port     = var.scheduler_port
 }
@@ -101,7 +101,7 @@ module "monitoring" {
   security_group_ids = [aws_security_group.internal.id]
   key_name           = var.key_name
   scheduler_ip       = module.scheduler.private_ip
-  instance_type      = "t3.medium"
+  instance_type      = "r5.large"
 }
 
 // Serratus-dl
@@ -109,7 +109,7 @@ module "download" {
   source             = "../worker"
 
   desired_size       = 0
-  max_size           = 200
+  max_size           = 250
 
   dev_cidrs          = var.dev_cidrs
   security_group_ids = [aws_security_group.internal.id]
@@ -134,7 +134,

In [12]:
# Initialize terraform
TF=$SERRATUS/terraform/main
cd $TF
terraform init



Initializing modules...

Initializing the backend...

Initializing provider plugins...

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


In [13]:
# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve

module.merge.data.aws_region.current: Refreshing state...
module.download.data.aws_availability_zones.all: Refreshing state...
module.merge.data.aws_ami.amazon_linux_2: Refreshing state...
module.scheduler.data.aws_region.current: Refreshing state...
module.merge.data.aws_availability_zones.all: Refreshing state...
module.download.data.aws_ami.amazon_linux_2: Refreshing state...
module.align.data.aws_ami.amazon_linux_2: Refreshing state...
module.download.data.aws_region.current: Refreshing state...
module.align.data.aws_availability_zones.all: Refreshing state...
module.align.data.aws_region.current: Refreshing state...
module.monitoring.data.aws_ami.ecs: Refreshing state...
module.scheduler.data.aws_ami.amazon_linux_2: Refreshing state...
module.align.module.iam_role.aws_iam_role.role: Creating...
module.download.module.iam_role.aws_iam_role.role: Creating...
module.monitoring.aws_iam_role.task_role: Creating...
module.download.aws_cloudwatch_log_group.g: Creating...
module.monitorin

## Running Serratus 
Upload the run data, scale-out the cluster, monitor performance.


### Run Monitors & Upload table
Open SSH tunnels to monitor node then open monitors in browser


In [14]:
cd $TF

# Open SSH tunnels to the monitor
./create_tunnels.sh

# If you get an error on port
# run:
# ps aux | grep ssh
# sudo kill <PID of SSH>
#


Tunnels created:

    localhost:3000 -- grafana

    localhost:9090 -- prometheus

    localhost:8000 -- scheduler


### Batch 14 - Cleanup


In [15]:
# Load SRA Run Info into scheduler ===================
# Scheduler DNS: 
echo "Loading SRARunInfo into scheduler "
echo "  File: $CURRENT_BATCH"
echo "  md5 : $(md5sum $WORK/$CURRENT_BATCH)"
echo "  date: $(date)"

curl -s -X POST -T $WORK/$CURRENT_BATCH localhost:8000/jobs/add_sra_run_info/

Loading SRARunInfo into scheduler 
  File: zoonotic_SraRunInfo_Batch14.csv
  md5 : 8de3442322c14486692c229bca0b45c9  /home/artem/serratus/notebook/200505_ab/zoonotic_SraRunInfo_Batch14.csv
  date: Wed May 13 12:30:46 PDT 2020
{"inserted_rows":5609,"total_rows":5609}


In [16]:
# Set Cluster Parameters =============================
cd $TF
# Make local changes to config file
echo "  Cluster Config File: "
cat serratus-config.json
echo ""
echo ""
# Re-upload config file
curl -T serratus-config.json localhost:8000/config

  Cluster Config File: 
{
"ALIGN_ARGS":"--very-sensitive-local",
"ALIGN_SCALING_CONSTANT":0.75,
"ALIGN_SCALING_ENABLE":true,
"ALIGN_SCALING_MAX":400,
"CLEAR_INTERVAL":777,
"DL_ARGS":"",
"DL_SCALING_CONSTANT":0.1,
"DL_SCALING_ENABLE":true,
"DL_SCALING_MAX":100,
"GENOME":"cov2m",
"MERGE_ARGS":"",
"MERGE_SCALING_CONSTANT":0.1,
"MERGE_SCALING_ENABLE":true,
"MERGE_SCALING_MAX":3,
"SCALING_INTERVAL":305,
"VIRTUAL_ASG_MAX_INCREASE":10,
"VIRTUAL_SCALING_INTERVAL":60
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0{"ALIGN_ARGS":"--very-sensitive-local","ALIGN_SCALING_CONSTANT":0.75,"ALIGN_SCALING_ENABLE":true,"ALIGN_SCALING_MAX":400,"CLEAR_INTERVAL":777,"DL_ARGS":"","DL_SCALING_CONSTANT":0.1,"DL_SCALING_ENABLE":true,"DL_SCALING_MAX":100,"GENOME":"cov2m","MERGE_ARGS":"","MERGE_SCALING_CONSTANT":0.1,"MERGE_S

### Error Handling


In [ ]:
# Error fixes (manually help along)

# ssh <scheduler IPv4>
# sudo docker ps
# sudo docker exec -it <container> bash
# apt install sqlite3 awscli

### ACCESSION OPERATIONS

# Reset SPLITTING accessions to NEW
# sqlite3 instance/scheduler.sqlite 'UPDATE acc SET state = "new" WHERE state = "splitting";'

# Reset SPLIT_ERR accessions to NEW
# (repeated failures can be missing SRA data)
# sqlite3 instance/scheduler.sqlite 'UPDATE acc SET state = "new" WHERE state = "split_err";'

# Reset MERGE_ERR accessions to MERGE_WAIT
# sqlite3 instance/scheduler.sqlite 'UPDATE acc SET state = "merge_wait" WHERE state = "merge_err";'

# Clear DONE Accessions (ONLY ON COMPLETION)
# sqlite3 instance/scheduler.sqlite 'DELETE FROM acc WHERE state = "merge_done";'

### BLOCK OPERATIONS

# Reset FAIL blocks to NEW
# sqlite3 instance/scheduler.sqlite 'UPDATE blocks SET state = "new" WHERE state = "fail";'

# Reset ALIGNING blocks to NEW
# sqlite3 instance/scheduler.sqlite 'UPDATE blocks SET state = "new" WHERE state = "aligning";'


## Shutting down procedures

Closing up shop.

In [17]:
# Dump the Scheduler SQLITE table to a local file
date
curl localhost:8000/db > \
  $WORK/zoonotic_batch14B_complete.sqlite

Wed May 13 20:20:43 PDT 2020
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 24.7M  100 24.7M    0     0   826k      0  0:00:30  0:00:30 --:--:-- 1323k


## Destroy Cluster

Close out all resources with terraform (will take a few minutes).


In [19]:
terraform destroy -auto-approve
# WARNING this will also delete the standard output bucket/data
# Save data prior to destroy



module.align.data.aws_ami.amazon_linux_2: Refreshing state...
module.merge.aws_cloudwatch_log_group.g: Refreshing state... [id=serratus-merge]
module.monitoring.data.aws_ami.ecs: Refreshing state...
module.download.data.aws_availability_zones.all: Refreshing state...
module.scheduler.data.aws_region.current: Refreshing state...
module.download.aws_cloudwatch_log_group.g: Refreshing state... [id=serratus-dl]
module.download.data.aws_ami.amazon_linux_2: Refreshing state...
module.merge.data.aws_region.current: Refreshing state...
module.scheduler.data.aws_ami.amazon_linux_2: Refreshing state...
module.merge.module.iam_role.aws_iam_role.role: Refreshing state... [id=SerratusIamRole-serratus-merge]
module.scheduler.aws_cloudwatch_log_group.scheduler: Refreshing state... [id=scheduler]
module.merge.data.aws_ami.amazon_linux_2: Refreshing state...
module.scheduler.module.iam_role.aws_iam_role.role: Refreshing state... [id=SerratusIamRole-scheduler]
module.merge.data.aws_availability_zones.

### Run Notes



717 or about 1% of total libraries remain unfinished.